In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc
import itertools
import importlib
from IPython.display import clear_output

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay
import bokeh.palettes as bkpalettes
import bokeh.util.hex as bkhex
import bokeh.transform as bktrfm
import bokeh.colors as bkcolors


# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp

# BBStudies
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.XMask.Utils as xutils
import BBStudies.Physics.Constants as cst
import BBStudies.Plotting.Bokeh.Tools as bktools
import BBStudies.Plotting.Bokeh.Presets as bkpresets
import BBStudies.Physics.Base as phys

bk.output_notebook()
def plot_phase_space(df,rfbucket = None):
    _FIGS   = {}
    _default_fig_width  = 1400
    _bot_tab_height     = 400
    padding             = 20 
    # Phase space plots:
    #=====================================
    adjustment = 0
    _FIGS['x-px'] = bkpresets.make_scatter_fig(df,xy=('x_sig','px_sig')     ,alpha=0.2,title=f'x norm. phase space ({len(df)} particles)'   ,width=int(_default_fig_width/3.5)+adjustment,height=_bot_tab_height,padding=padding)
    _FIGS['y-py'] = bkpresets.make_scatter_fig(df,xy=('y_sig','py_sig')     ,alpha=0.2,title='y norm. phase space'                          ,width=int(_default_fig_width/3.5),height=_bot_tab_height,padding=padding)
    _FIGS['zeta-pzeta'] = bkpresets.make_scatter_fig(df,xy=('zeta','pzeta') ,alpha=0.2,title='zeta phase space'                             ,width=int(_default_fig_width/3.5),height=_bot_tab_height,padding=padding)


    _FIGS['x-px'].min_border_left  = padding
    bktools.set_aspect(_FIGS['x-px']       , x_lim=(-11,11),y_lim=(-11,11), aspect=1, margin=padding-adjustment)
    bktools.set_aspect(_FIGS['y-py']       , x_lim=(-11,11),y_lim=(-11,11), aspect=1, margin=0)
    # bktools.set_aspect(_FIGS['zeta-pzeta'] , x_lim=(-1,1),y_lim=(-1,1), aspect=1, margin=0)
    _FIGS['zeta-pzeta'].x_range  = bkmod.Range1d(-0.5, 0.5)
    _FIGS['zeta-pzeta'].y_range  = bkmod.Range1d(-5e-4, 5e-4)

    _FIGS['x-px'].xaxis.axis_label = r'$$\tilde x /\sqrt{\varepsilon_{x}}$$'
    _FIGS['x-px'].yaxis.axis_label = r'$$\tilde p_x /\sqrt{\varepsilon_{x}}$$'

    _FIGS['y-py'].xaxis.axis_label = r'$$\tilde y /\sqrt{\varepsilon_{y}}$$'
    _FIGS['y-py'].yaxis.axis_label = r'$$\tilde p_y /\sqrt{\varepsilon_{y}}$$'

    _FIGS['zeta-pzeta'].xaxis.axis_label = r'$$\zeta$$'
    _FIGS['zeta-pzeta'].yaxis.axis_label = r'$$p_\zeta $$'

    if rfbucket is not None:
        color = 'lightseagreen'
        ls    = 'solid'
        label = f'RF Bucket'
        for zcut in list(np.linspace(0.001,rfbucket.zeta_max,10)) + [0.09]:
            zeta_vec,delta_vec = rfbucket.invariant(zcut,npoints = 1000)

            if zcut == 0.09:
                color = 'mediumvioletred'
                ls    = 'dotted'
                label = f'1 σ'
            
            line_top = _FIGS['zeta-pzeta'].line(x=zeta_vec,y=delta_vec, line_width=2, color=color, alpha=0.2, line_dash=ls, legend_label=label)
            line_bot = _FIGS['zeta-pzeta'].line(x=zeta_vec,y=-delta_vec, line_width=2, color=color, alpha=0.2, line_dash=ls, legend_label=label)
            line_top.level = 'underlay'
            line_bot.level = 'underlay'
            # plt.plot(zeta_vec,delta_vec,'-',color='C2')
            # plt.plot(zeta_vec,-delta_vec,'-',color='C2')
    


    grid = bklay.gridplot([[_FIGS['x-px'] ,_FIGS['y-py'] ,_FIGS['zeta-pzeta']]],toolbar_location='right')
    #=====================================
    return grid


Loading BokehJS ...

In [ ]:
import importlib
importlib.reload(bkpresets)

---
# Loading collider for RFBucket
---

In [2]:
collider_path   = 'colliders/collider_BUNCH_0000.json'
collider        = xt.Multiline.from_json(collider_path)
line            = collider['lhcb1']
line.build_tracker()
rfbucket        = xPlus.RFBucket(line)
clear_output()

---
# Generating particles
---

In [ ]:



density     = 363/(2*np.pi*1)
_generators = [pd.DataFrame({'x_sig':[0],'px_sig':[0]})]
for r in np.linspace(1,10,10):
    n_points = int(2*np.pi*r*density)
    _df      = phys.polar_grid( r_sig     = [r],
                                theta_sig = np.linspace(0,2*np.pi,n_points))
    _generators.append(_df.rename(columns={'y_sig':'px_sig'})[['x_sig','px_sig']])


generator = pd.concat(_generators).reset_index()
# generator = generator.rename(columns={'y_sig':'px_sig'})[['x_sig','px_sig']]
# generator = pd.concat([pd.DataFrame({'x_sig':[0],'px_sig':[0]}),generator]).reset_index() # Adding on axis particle
n_part = len(generator)



#distribute longitudinal
# zeta_vec,delta_vec = rfbucket.invariant(0.09,npoints = 10000)
# zeta_vec,delta_vec = redistribute_along(zeta_vec,delta_vec,n_points = n_part)
# zeta_vec,delta_vec = redistribute_along(list(zeta_vec) + list(zeta_vec[::-1])[:-2],list(delta_vec) + list(-delta_vec[::-1])[:-2],n_points = n_part)


# Initializing:
#======================================================
coord_df = pd.DataFrame({   'x_sig' :np.zeros(n_part),
                            'px_sig':np.zeros(n_part),
                            'y_sig' :np.zeros(n_part),
                            'py_sig':np.zeros(n_part),
                            'zeta'  :np.zeros(n_part),
                            'delta' :np.zeros(n_part)})
#======================================================


# Overwriting
coord_df.loc[:,'y_sig']  = generator.x_sig.values
coord_df.loc[:,'py_sig'] = generator.px_sig.values
coord_df.loc[:,'zeta']   = 2*0.09
# coord_df.loc[:,'delta'] = delta_vec

# Plotting
coord_df.insert(5,'pzeta',xPlus.delta2pzeta(coord_df['delta'],beta0=1))
bk.show(plot_phase_space(coord_df,rfbucket = rfbucket))
display(coord_df)

In [ ]:
coord_df.insert(0,'particle',coord_df.index)
particle_id = coord_df.index

bk.show(plot_phase_space(coord_df[coord_df.particle.isin(particle_id[::100])],rfbucket = rfbucket))

---
# Saving Results
---

In [ ]:
coord_df.to_parquet('particles/YPLANE_ZETA_2.parquet')

In [ ]:
coord_df.to_parquet('particles/XPLANE_ZETA_0.parquet')

---
# Load and plot
---

In [ ]:
coord_df = xPlus.import_parquet('particles/XPLANE_ZETA_1.parquet')
bk.show(plot_phase_space(coord_df,rfbucket = None))
display(coord_df)